# Data 515 LIME
_Local Interpretable Model-agnostic Explanations_

Spring 2019

__Instructors:__
Joseph L. Hellerstein (jlheller@uw.edu)

David Beck (dacb@uw.edu)

Bernease Herman (bernease@uw.edu)

Sam Gao (gaoz6@cs.washington.edu)

## Component Specification

This component specification covers the proposed final project for the LIME team:

Francisco Javier Salido Magos (javiers@uw.edu)

Patrick King (pking70@uw.edu)

Suman Bhagavathula (sumanbh@uw.edu)

For a more general explanation of LIME, as well as links to the code and original paper by LIME's authors, you can go here: https://www.oreilly.com/learning/introduction-to-local-interpretable-model-agnostic-explanations-lime

### Software components

Our project has three major modules, each of which can be broken down into multiple components. The three modules are:

- The datasets module has two sub modules, one for data acquisition and another for clean and merge to get the final dataset used for training and analysis.
- Machine Learning (ML) model that takes a number of predictors as input, including college name, student major, type of school and other, and outputs a likely graduating salary bracket for students from that college/major combination.
- A LIME module that can help a data scientist understand which predictors played a major role in defining the ML model's output for a particular set of inputs.

#### _Data Sets_

The first submodule in the datasets module contains two functions, the first function acquires the most recent cohorts dataset and the second function acquires the salaries by region dataset. These two functions do not take any input. The output will be a pandas dataframe for the corresponding dataset loaded from the data directory. 

The second submodule in this module performs the following operations: normalized the College names columns in both the datasets, joins the two datasets based on this normalized college name, retains a set of hand selected features, generates a new column that has Median mid career salary categorized into three classes to be used as the Classification target. The implementation consists of two functions, one to clean and merge the data sets and the other function which invokes this cleaner function to get the final prepared pandas dataset to pass it on to the caller.


#### _Machine Learning Model_

The machine learning module contains three functions, each of which enables us to train and test a specific scikitlearn ML model. The three models are multiclass logistic regression, random forest and decision tree. They load the dataset using the load dataset function in the datasets module, scale the inputs to values between 0 and 1, split the dataset into training and testing and then fit the corresponding machine learning algorithm. They take one optional parameter called print_score that can be used to specify whether or not to print the testing accuracy score after model training and using the predict function on the test set. The output of the function is the model object itself.

#### _LIME Module_

The LIME module will approximate the underlying ML model with an interpretable one that is learned on randomly perturbed samples of an input/output instance of the underlying model. For purposes of this discussion we shall heretofore refer to the underlying ML model as the "complex" model, and to the LIME model as the "simple" model.

The main idea of LIME is that we have some complex classifier, a random forest, boosted trees or a neural network, the workings of which cannot be explained in terms a human being can understand. We say that the complex ML model $f(x)$ is not _interpretable_ if, given a set of input predictors $x = (x_1, x_2, ...., x_m)$, an output $f(x)$ that is either a value indicating the probability that $x$ belongs to a certain class, or a label designating that class, the reason why a particular $x_0$ results in a particular output $f(x_0)$ cannot be understood by a person, even if all the details of the model are known. Since the global workings of the complex model cannot be explained in human terms, we use simpler linear models to explain its local behavior in the vicinity of our chosen $(x, f(x))$.

For purposes of this project we will limit the discussion to Machine Learning models for which the individual input predictors can be understood by humans, they are not engineered features, and that can be represented in tabular form.

Inputs to the LIME module should be:

- The single instance $x$, and corresponding complex model output $f(x)$ that we wish to interpret.
- A description of the domain and probability distribution for each normalized predictor (mean $m = 0$ and standard deviation $\sigma = 1$), encoded in the form of one histogram for each predictor in the model.
- Access to the complex model itself, which we will use to compute labels for a number of perturbed samples that will be generated by the module.

The LIME module will have two main components:

The first component, $lime\_sample$ will generate random samples or perturbations that will simulate real input data. Inputs will be the number of samples we wish to generate and the data used to train the ML model. Training data is required so that the probability distribution of random samples will match that of the actual data. The output will be a set of $n$ randomized samples of the complex model's behavior. Samples are generated by randomly chosing a value from each predictor/histogram, to form a randomized instance $z_i = (z_{i,1}, z_{i,2}, ...., z_{i,m})$, which is then provided as input to the complex model to obtain the corresponding output $f(z_i)$. The total number of randomized samples we generate, $n$, should be large, and these samples will be distributed all over the sample space. Note that we'll need to options for this first component, one that will generate random samples for attributes that have discrete values, and a second for continous values.

The second component, $lime\_fit$, will take the set of randomized samples generated above as input, and output a linear model, the simple model, that describes the behavior of the complex model in the vicinity of the input instance $x$. To do this, $lime\_fit$ will need to execute two steps:

1.- Compute a proximity measure/weight for each random sample. Since $lime\_fit$ produces a set of randomized samples that are distributed across the entire sample space, we'll use this proximity measure $\pi_x(z_i)$ to penalize the samples based on how distant they are from $x$. The further away $z_i$ is from $x$, the greater the penalty:

$$\pi_x(z_i) = \exp^{-\frac{D(x,z_i)^2}{\sigma^2}}$$ 

where $D(x,z_i)$ is the L2 distance: 

$$D(x,z_i)^2 = (x_1 - z_{i,1})^2 + (x_2 - z_{i,2})^2 + .... + (x_m - z_{i,m})^2$$

2.- Fit a linear model using the LASSO algorithm, with $x$ and the set of perturbed samples $z_1, z_2, ...., z_n$ as inputs. Thus, we wish to find:

$$arg min_g \sum_{i} \pi_x(z_i)(f(z_i) - wz_i)^2 + \lambda|w|$$ 

where: 

$$g(z_i) = w_1 z_{i,1} + w_2z_{i,2} + .... + w_mz_{i,m}$$

and $w$ is the set of coefficients $w_1, w_2, ...., w_m$ assigned by the linear model to each predictor.

To fit the model we must employ the shrinkage method LASSO, as it shrinks the magnitued of those linear regression coefficients $w$ that are less correlated to the model's output, effectively helping us reduce the complexity of the simple model by reducing the number of non-zero predictors it uses.

The output will be the intercept and coefficients of the linear model fitted using LASSO. This model will facilitate interpretation of the complex ML model's behavior in the locality of the selected input instance.

### MVC architecture

The modules mentioned in the above section are organized into a Model View Controller architecture, with a folder for each of these components. As the name suggests, the Model contains all of the modules and functions for data acquisition and cleaning, the Controller contains modules and functions that perform various business operations on the data and the View contains all of the modules and functions for presenting the data and analysis results. The structure is shown below for the current project:

Model:
- get_college_datasets (return the individual datasets)
- load_college_datasets (clean and join the datasets, return the final dataset ready for processing)

Controller:
- fit_sklearn_models (apply the three different scikitlearn models on the dataset and return the model object)
- LIMEaid (contains the custom implementations for LIME functionality)

View:
- To be done.

### Interactions to accomplish use cases

The interaction is quite simple. The first module is the ML model that the data scientist needs to understand. Since the model is complex, direct interpretation of its output would require too much effort or is simply not possible in human terms.

In order to gain insight, the data scientist needs to develop intuition on how the predictors at the input of the model affect its output classification, for each test instance. Given that we are dealing with a complex ML model, we use LIME to provide local explanations that will enable the data scientist to acquire intuition of what drive's the model's decisions in the vicinity of each specific test instance that are to be interpreted.

### Preliminary plan

Our preliminary plan consists of a two-pronged approach:

The Data and Machine Learning modules:

- Inspection of the datasets we plan to employ.
- Implementation of the data_cleanse component.
- Fitting of a number of ML models to classify colleges and majors into potential salary brackets.

For the LIME module:

- Do additional research on some of the mathematical and logistic subtleties of sampling and optimization for LIME.
- Build and test the lime_sampler.
- Build and test the lime_fit.
- Build and test the lime_result.
- Do end-to-end testing of the components.

Once the above steps have been completed, we will evaluate the outputs of the ML model(s) using LIME and produce the final report.
